In [1]:
pip install nashpy

  Created wheel for nashpy: filename=nashpy-0.0.19-cp36-none-any.whl size=10815 sha256=d82eb30fddc24e8628087b8a1d19b756582c96c644f076fc3470bc8dc8c78db0
  Stored in directory: /root/.cache/pip/wheels/18/e9/56/2d04d01a6969d167f86d3afcb3d128c0b43d0d73ea471c835b
Successfully built nashpy


In [2]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving variant23.csv to variant23.csv


In [20]:
import nashpy as nash
import numpy as np
import pandas as pd
df = pd.read_csv('variant23.csv')
df

,b0,b1,b2,b3,b4
0,2,9,3,7,5
1,4,10,3,6,5
2,5,11,3,7,2
3,3,6,10,5,11
4,3,6,3,5,3


In [21]:
def minrow(row):
    return row.min()
desc = df.describe()
new_df = df.copy()
new_df['amin'] = new_df.apply(minrow, axis=1)
new_df.loc['bmax'] = desc.loc['max']
new_df

,b0,b1,b2,b3,b4,amin
0,2.0,9.0,3.0,7.0,5.0,2.0
1,4.0,10.0,3.0,6.0,5.0,3.0
2,5.0,11.0,3.0,7.0,2.0,2.0
3,3.0,6.0,10.0,5.0,11.0,3.0
4,3.0,6.0,3.0,5.0,3.0,3.0
bmax,5.0,11.0,10.0,7.0,11.0,NaN


In [22]:
bmax = new_df.loc['bmax'].min()
amin = new_df['amin'].max()
bmaxid = new_df.loc['bmax'].idxmin()
aminid = new_df['amin'].idxmax()
print('Сідлова точка:',new_df[bmaxid][aminid])
print(f'Стовбець: {bmaxid}, Рядок: {aminid}')
if bmax != amin:
    print('Нижня ціна гри:',amin)
    print('Верхня ціна гри:',bmax)
    print(f'Гра не має вирішення в чистих стратегій: {bmax} != {amin}')

Сідлова точка: 4.0
Стовбець: b0, Рядок: 1
Нижня ціна гри: 3.0
Верхня ціна гри: 5.0
Гра не має вирішення в чистих стратегій: 5.0 != 3.0


In [23]:
# Знаходження домінуючих рядків
print('Знаходження домінуючих рядків:')
n = df.shape[0]
adellist = set()
for i in range(n): 
    for j in range(n):
        if i!=j and (df.loc[i]>=df.loc[j]).sum() == 5:
            print(f'{list(df.loc[i])} домінує над ,{list(df.loc[j])}')
            adellist.add(j)
# знаходження домінуючих стовпців
print('Знаходження домінуючих стовпців:')
n = df.shape[1]
bdellist = set()
for i in df.columns: 
    for j in df.columns:
        if i!=j and (df[i]<=df[j]).sum() == 5:
            print(f'{list(df[i])} домінує над ,{list(df[j])}')
            bdellist.add(j)
            
df.drop(adellist,inplace = True)
df.drop(bdellist,axis = 1,inplace = True)
df

Знаходження домінуючих рядків:
[4, 10, 3, 6, 5] домінує над ,[3, 6, 3, 5, 3]
[3, 6, 10, 5, 11] домінує над ,[3, 6, 3, 5, 3]
Знаходження домінуючих стовпців:
[2, 4, 5, 3, 3] домінує над ,[9, 10, 11, 6, 6]
[2, 4, 5, 3, 3] домінує над ,[7, 6, 7, 5, 5]
[7, 6, 7, 5, 5] домінує над ,[9, 10, 11, 6, 6]


,b0,b2,b4
0,2,3,5
1,4,3,5
2,5,3,2
3,3,10,11


In [24]:
df2 = df.copy()
desc2 = df2.describe()
df2['amin'] = df2.apply(minrow, axis=1)
df2.loc['bmax'] = desc2.loc['max']
df2

,b0,b2,b4,amin
0,2.0,3.0,5.0,2.0
1,4.0,3.0,5.0,3.0
2,5.0,3.0,2.0,2.0
3,3.0,10.0,11.0,3.0
bmax,5.0,10.0,11.0,NaN


In [25]:
arr = pd.read_csv('variant23.csv').to_numpy()
arr

array([[ 2,  9,  3,  7,  5],
       [ 4, 10,  3,  6,  5],
       [ 5, 11,  3,  7,  2],
       [ 3,  6, 10,  5, 11],
       [ 3,  6,  3,  5,  3]])

In [26]:
game = nash.Game(arr)
list2 = list(game.lemke_howson(initial_dropped_label=0))
print('Оптимальні змішані стратегії гравця А:')
print(list2[0])
print('Оптимальні змішані стратегії гравця B:')
print(list2[1])
print('Ціна гри:')
print(game[list2[0],list2[1]][0])

Оптимальні змішані стратегії гравця А:
[0.         0.         0.72727273 0.27272727 0.        ]
Оптимальні змішані стратегії гравця B:
[0.81818182 0.         0.         0.         0.18181818]
Ціна гри:
4.454545454545454
